---
title: "Labelling contiguous blocks"
author: "Damien Martin"
date: "2025-06-04"
categories: [pandas, sql]
---

# Problem

Something I have to do a reasonably frequent basis is find the "changepoints" in data, or when a particular field changed value. More precisely, we want to get the first row when each _contiguous_ block of values in a column.

For example, you can image that a table contains the following information about an item:

- it's price that day
- how much of a discount it was on
- the date
- the SKU

If we want to turn this from a series of snapshots to an event table, how would we do that?

In [14]:
#| echo: false

import pandas as pd 

items = pd.DataFrame([
    {'sku_id': 'sk001', 'date': '2025-01-01', 'discount': 0, 'base_price': 120},
    {'sku_id': 'sk001', 'date': '2025-01-02', 'discount': 0, 'base_price': 120},
    {'sku_id': 'sk001', 'date': '2025-01-03', 'discount': 0, 'base_price': 120},
    {'sku_id': 'sk001', 'date': '2025-01-04', 'discount': 0.1, 'base_price': 120},
    {'sku_id': 'sk001', 'date': '2025-01-05', 'discount': 0.1, 'base_price': 120},
    {'sku_id': 'sk001', 'date': '2025-01-06', 'discount': 0.1, 'base_price': 120},
    {'sku_id': 'sk001', 'date': '2025-01-07', 'discount': 0.4, 'base_price': 120},
    {'sku_id': 'sk001', 'date': '2025-01-08', 'discount': 0, 'base_price': 120},
    {'sku_id': 'sk001', 'date': '2025-01-09', 'discount': 0, 'base_price': 120},
    {'sku_id': 'sk002', 'date': '2025-01-01', 'discount': 0.3, 'base_price': 90},
    {'sku_id': 'sk002', 'date': '2025-01-02', 'discount': 0.3, 'base_price': 90},
    {'sku_id': 'sk002', 'date': '2025-01-03', 'discount': 0.3, 'base_price': 90},
    {'sku_id': 'sk002', 'date': '2025-01-04', 'discount': 0.5, 'base_price': 90},
    {'sku_id': 'sk002', 'date': '2025-01-05', 'discount': 0.5, 'base_price': 90},
    {'sku_id': 'sk002', 'date': '2025-01-06', 'discount': 0.5, 'base_price': 90},
    {'sku_id': 'sk002', 'date': '2025-01-07', 'discount': 0.5, 'base_price': 90},
    {'sku_id': 'sk002', 'date': '2025-01-08', 'discount': 0.6, 'base_price': 90},
    {'sku_id': 'sk002', 'date': '2025-01-09', 'discount': 0.6, 'base_price': 90},
]).assign(price=lambda x:x.base_price*(1-x.discount)).drop('base_price',axis=1)

items

,sku_id,date,discount,price
0,sk001,2025-01-01,0.0,120.0
1,sk001,2025-01-02,0.0,120.0
2,sk001,2025-01-03,0.0,120.0
3,sk001,2025-01-04,0.1,108.0
4,sk001,2025-01-05,0.1,108.0
5,sk001,2025-01-06,0.1,108.0
6,sk001,2025-01-07,0.4,72.0
7,sk001,2025-01-08,0.0,120.0
8,sk001,2025-01-09,0.0,120.0
9,sk002,2025-01-01,0.3,63.0


What we want instead is 

In [15]:
#| echo: false

items2 = items.copy()
new_group_mask = items2.groupby('sku_id')['discount'].transform(lambda x: x!=x.shift())
items2[new_group_mask]

,sku_id,date,discount,price
0,sk001,2025-01-01,0.0,120.0
3,sk001,2025-01-04,0.1,108.0
6,sk001,2025-01-07,0.4,72.0
7,sk001,2025-01-08,0.0,120.0
9,sk002,2025-01-01,0.3,63.0
12,sk002,2025-01-04,0.5,45.0
16,sk002,2025-01-08,0.6,36.0


Note that when `sk001` returned to full price, we get a new entry -- this is not as simple as grouping by the price or the discount columns. 

An alternative way we might want to present this is as a series of periods of time:

In [13]:
#| echo: false

items2 = items.copy()
items2['is_new_group'] = items2.groupby('sku_id')['discount'].transform(lambda x: x!=x.shift())
items2['group_id'] = items2.is_new_group.cumsum()
items2.groupby(['sku_id', 'group_id', 'discount', 'price']).agg(
    start_date=('date', 'min'),
    end_date=('date', 'max'),
).reset_index().drop('group_id', axis=1)

,sku_id,discount,price,start_date,end_date
0,sk001,0.0,120.0,2025-01-01,2025-01-03
1,sk001,0.1,108.0,2025-01-04,2025-01-06
2,sk001,0.4,72.0,2025-01-07,2025-01-07
3,sk001,0.0,120.0,2025-01-08,2025-01-09
4,sk002,0.3,63.0,2025-01-01,2025-01-03
5,sk002,0.5,45.0,2025-01-04,2025-01-07
6,sk002,0.6,36.0,2025-01-08,2025-01-09


# Solution

## Pandas

In [16]:
items2 = items.copy()
new_group_mask = items2.groupby('sku_id')['discount'].transform(lambda x: x!=x.shift())
items2[new_group_mask]

,sku_id,date,discount,price
0,sk001,2025-01-01,0.0,120.0
3,sk001,2025-01-04,0.1,108.0
6,sk001,2025-01-07,0.4,72.0
7,sk001,2025-01-08,0.0,120.0
9,sk002,2025-01-01,0.3,63.0
12,sk002,2025-01-04,0.5,45.0
16,sk002,2025-01-08,0.6,36.0


In [17]:
items2 = items.copy()
items2['is_new_group'] = items2.groupby('sku_id')['discount'].transform(lambda x: x!=x.shift())
items2['group_id'] = items2.is_new_group.cumsum()
items2.groupby(['sku_id', 'group_id', 'discount', 'price']).agg(
    start_date=('date', 'min'),
    end_date=('date', 'max'),
).reset_index().drop('group_id', axis=1)

,sku_id,discount,price,start_date,end_date
0,sk001,0.0,120.0,2025-01-01,2025-01-03
1,sk001,0.1,108.0,2025-01-04,2025-01-06
2,sk001,0.4,72.0,2025-01-07,2025-01-07
3,sk001,0.0,120.0,2025-01-08,2025-01-09
4,sk002,0.3,63.0,2025-01-01,2025-01-03
5,sk002,0.5,45.0,2025-01-04,2025-01-07
6,sk002,0.6,36.0,2025-01-08,2025-01-09
